## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [34]:
from dotenv import load_dotenv                                   # Load environment variables from .env file
from agents import Agent, Runner, trace, function_tool           # Import core agent framework components
from openai.types.responses import ResponseTextDeltaEvent        # OpenAI streaming response type
from typing import Dict                                          # Python type hints
import sendgrid                                                  # SendGrid email service SDK
import os                                                        # Operating system interface
from sendgrid.helpers.mail import Mail, Email, To, Content       # SendGrid email components
import asyncio                                                   # Async I/O support for Python
import requests                                                  # HTTP requests library
import certifi
os.environ['SSL_CERT_FILE'] = certifi.where()

In [2]:
load_dotenv(override=True)

True

In [35]:
# Let's just check emails are working for you

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("federico.tognetti@gmail.com")  # Change to your verified sender
    to_email = To("federico.tognetti@gmail.com")  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202


### Did you receive the test email

If you get a 202, then you're good to go!

#### Certificate error

If you get an error SSL: CERTIFICATE_VERIFY_FAILED then students Chris S and Oleksandr K have suggestions:  
First run this: `!uv pip install --upgrade certifi`  
Next, run this:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

#### Other errors or no email

If there are other problems, you'll need to check your API key and your verified sender email address in the SendGrid dashboard

Or use the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

(Or - you could always replace the email sending code below with a Pushover call, or something to simply write to a flat file)

In [ ]:
# # NB: This code will allow to use Pushover to send emails

# pushover_user = os.getenv("PUSHOVER_USER")
# pushover_token = os.getenv("PUSHOVER_TOKEN")
# pushover_url = "https://api.pushover.net/1/messages.json"


# def send_test_email(message):
#     print(f"Push: {message}")
#     payload = {"user": pushover_user, "token": pushover_token, "message": message}
#     requests.post(pushover_url, data=payload)

# send_test_email("This is a test email")

Push: This is a test email


## Step 1: Agent workflow

In [36]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [37]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [ ]:

# This code runs the sales agent and streams its response in real-time:
# 1. Calls the sales agent with "Write a cold sales email" input
# 2. Asynchronously iterates through the response stream events
# 3. When a text response event is received, prints it immediately without line breaks
# The flush=True ensures the output appears in real-time rather than being buffered

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Streamline Your SOC 2 Compliance with ComplAI

Dear [Recipient's Name],

I hope this message finds you well.

Navigating the complexities of SOC 2 compliance can often be overwhelming and time-consuming. At ComplAI, we understand the challenges companies face in maintaining compliance and preparing for thorough audits. That’s why we’ve developed an AI-powered SaaS tool designed to simplify and streamline the entire process.

Our solution offers:

- **Automated Documentation**: Easily generate and manage the necessary documentation for SOC 2 compliance.
- **Real-Time Monitoring**: Stay updated on your compliance status with instant alerts and insights.
- **Comprehensive Audit Preparation**: Simplify your audit processes with organized reports and evidence collection.

By leveraging our platform, organizations like yours have improved their compliance efficiency, reduced audit prep time, and mitigated risks.

I would love to schedule a brief call to discuss how ComplAI can suppo

In [ ]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    # .gather is used to run the agents in parallel making multiple calls to the LLM
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results] # list comprehension to get the final output of each agent

for output in outputs:
    print(output + "\n\n")


Subject: Simplify Your SOC 2 Compliance Process with ComplAI

Dear [Recipient's Name],

I hope this message finds you well.

Navigating the complexities of SOC 2 compliance can be a daunting task, but it doesn't have to be. At ComplAI, we provide a cutting-edge SaaS solution designed to streamline your compliance efforts and prepare you for audits with ease. Our AI-powered platform automates time-consuming processes, ensuring that you not only meet compliance requirements but also maintain high standards of security and efficiency.

Here are a few ways ComplAI can benefit your organization:

- **Automated Reports**: Save countless hours with our intelligent reporting system, which generates comprehensive compliance documentation.
- **Real-time Monitoring**: Stay ahead of potential risks and compliance gaps with ongoing assessments that keep you informed.
- **Streamlined Audits**: Simplify your audit preparation and reduce stress with organized, easily retrievable documentation.

I woul

In [ ]:
# This is a new agent that picks the best cold sales email from the given options.
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.", # this instruction in very important to avoid LLM hallucinations and unwanted output
    model="gpt-4o-mini"
)

In [ ]:
# This code will chain two agents:
# 1. A sales agent that writes a cold sales email
# 2. A sales picker that picks the best cold sales email from the given options

# The sales picker is a new agent that picks the best cold sales email from the given options.
# It imagines to be a customer and pick the one most likely to respond to.
# It doesn't give an explanation; it replies with the selected email only.

message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results] # list comprehension to get the final output of each agent

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs) # join the outputs with a new line

    best = await Runner.run(sales_picker, emails) # run the sales picker on the emails to select the best one from the list of the previous agents

    print(f"Best sales email:\n{best.final_output}") # print the best sales email


Best sales email:
Subject: Is Your Compliance as Cool as Your Coffee?

Hey [Recipient's Name],

You know that moment when you think you've got your favorite coffee order down to a science, only to be hit with a surprise flavor? That’s kind of how audits feel without the right tools—in short, sheer chaos.

At ComplAI, we believe compliance shouldn't taste like burnt toast. Our AI-powered SaaS tool whips your SOC2 compliance into shape faster than you can say “audit prep,” leaving you with more time for the fun stuff—like exploring that elusive ‘work-life balance’ everyone keeps talking about.

Imagine a world where audits are stress-free and your compliance checks are smoother than a freshly brewed cappuccino. ☕ Sounds dreamy, right? With ComplAI, you can enjoy that blissful state, minus the caffeine jitters.

Curious about how we can brew up some compliance magic for your team? I’d love to schedule a quick chat—promise it'll be more invigorating than your morning espresso!

Looking for

Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [14]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [15]:
sales_agent1

Agent(name='Professional Sales Agent', instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [38]:
# This is a function tool that sends an email

@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
    to_email = To("ed.donner@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [ ]:
# This is a function tool that sends a Pushover notification - Alternative solution to Sendgrid

# @function_tool # decorator to make the function a function tool
# def send_pushover(body: str):
#     """ Send out a Pushover notification with the given message """
#     payload = {"user": pushover_user, "token": pushover_token, "message": body}
#     requests.post(pushover_url, data=payload)
#     return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [39]:
# Let's look at it
send_email

# IMPORTANT: since we have added @function_tool decorator, we can now use the function as a tool in the agent

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x115f151c0>, strict_json_schema=True, is_enabled=True)

### And you can also convert an Agent into a tool

In [40]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x115516480>, strict_json_schema=True, is_enabled=True)

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [41]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x115517e20>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x115f15f80>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required'

## And now it's time for our Sales Manager - our planning agent

In [ ]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""

# VERY IMPORTANT: here we create a sales manager agent that will use the tools to generate the best cold sales email
# The agent is crated with a name, instructions, tools and model to use
# This way this agent will be able to follow the instruction and use the tools to generate the best cold sales email
sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the SendGrid website, to hunt for clues. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [51]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [52]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("federico.tognetti@gmail.com")  # Change to your verified sender
    to_email = To("federico.tognetti@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

# Since Sendgrid is not working, we can use the function tool to send a Pushover notification

# @function_tool
# def send_pushover(message: str) -> Dict[str, str]:
#     """ Send out a Pushover notification with the given message """
#     payload = {"user": pushover_user, "token": pushover_token, "message": message}
#     requests.post(pushover_url, data=payload)
#     return {"status": "success"}

In [53]:
tools = [subject_tool, html_tool, send_html_email]

In [47]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x115f16840>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x115f16c00>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to all sales pros

In [54]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    # Description that will be shown to other agents when they hand off (delegate) work to this agent
    handoff_description="Convert an email to HTML and send it") 

# IMPORTANT: this handoff agent as 3 tools: subject_writer, html_converter and send_pushover

### Now we have 3 tools and 1 handoff

In [55]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x115517e20>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x115f15f80>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': 

In [56]:
# Improved instructions thanks to student Guillermo F.

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

## Anwer to question 1
The line that changed it from being an Agentic "workflow" is that we used a primary agent the possibility to use tools (other agents as tools) and one handof to decide how to better accomplish the task without using code to define individual tasks to be accomplished in sequence.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Answer to commercial implications
The best commercial implication are: 
* Selena AI to help financial advisor better accoplish their tasks saving time and making more money
* Domus revita to evaluate which potential house restructuring can have best potential and returns. Using AI Agents can speed up the initial instructorial phase

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.

In [72]:
# Excercise create an agent that evaluate the create an advertising for an house requalification project

copywriter_instructions = """
You are a copywriter that creates an advertising for an house requalification project with enthusiastic and compelling tone.
This advertising is aimed to owner of big old houses in central or qualitative areas of the city.
The owner is probably strugling to sell the house as it is since it's not in a good condition an a lot of money is needed to requalify it.
You must explain that our team has very qualified architects and contractors that can help you to requalify your house.
The old big house will be partitioned in two housing units and completelly restructured.
One will be sold by our team and one will remain as payment to the owner that will have a new requalified house to sell, rent or live in.
IMPORTANT: generate the advertising in italian using max 200 words.
"""

copywriter = Agent(
    name="Copywriter",
    instructions=copywriter_instructions,
    model="gpt-4o-mini"
)

copywriter_tool = copywriter.as_tool(tool_name="create_advertising", tool_description="Create an advertising for an house requalification project")

# Let's look at it
copywriter_tool

FunctionTool(name='create_advertising', description='Create an advertising for an house requalification project', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'create_advertising_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1164fc720>, strict_json_schema=True, is_enabled=True)

In [73]:
tools = [copywriter_tool]

advertiser_instructions = """
Use the tools to create an advertising for an house requalification project.
IMPORTANT: use the tools to create the advertising. DO NOT write the advertising yourself.
"""

advertiser = Agent(
    name="Advertiser",
    instructions=advertiser_instructions,
    tools=tools,
    model="gpt-4o-mini"
)

message = "Create an advertising for an house requalification project"

with trace("Automated SDR"):
    result = await Runner.run(advertiser, message)

print(result.final_output)

### Trasforma la tua casa in un capolavoro!

Possiedi una grande casa d'epoca nel cuore della città ma fatichi a venderla? La soluzione è a portata di mano! Il nostro progetto di riqualificazione è pensato per te. Con un team di architetti e contractor altamente qualificati, trasformeremo la tua dimora in un gioiello irresistibile.

Ridimensioneremo la tua casa in due unità abitative completamente ristrutturate: una sarà venduta dal nostro team e l'altra rimarrà a te. Avrai una nuova casa rinnovata che potrai vendere, affittare o semplicemente vivere!

Non lasciare che la tua proprietà continui a perdere valore. Siamo qui per aiutarti a ottenere il massimo dal tuo investimento, aumentando il comfort, l'efficienza energetica e, soprattutto, il valore del tuo immobile.

Contattaci oggi stesso per scoprire come possiamo trasformare il tuo sogno in realtà. Il tuo futuro inizia qui!
